## OVERVIEW

The project contains several simulators that were used for different parts of the paper. Essentially we have

__Deterministic simulator__

- `simtools.py` The numeric solver for the PDE that describes the evolution of the secondary resistance mechanisms over time.
- `abc.py` The approximate bayesian computation parameter fitting system, that, given some observation of gain and loss of a possible resistance mechanism parameter finds the probable cell growth rate as a function of that parameter.

__Stochastic simulators__
These take .hdf5 input files containing the adaptation parameter probability density function, and a parametrized rate function + some other simulation settings from `simulation-config.toml`. For the `statics-mpi` the probability density functions are just the initial conditions, but for `dap-mpi` and `holiday-mpi` the input has one or more time series of PDFs as calculated by the PDE.

- `dap-mpi` An implementation of Gillespie's algorithm simulating the growth of cells with the continuous secondary resistance mechanism. Can optionally use a time-varying death rate.
- `holiday-mpi` Same as above, but takes an entire set of PDE backgrounds, each for a different drug holiday.
- `statics-mpi` Constrained population size stochastic simulation variants. Specifically to test whether the ABC inferred parameters produce the same results (in terms of resistance parameter adaptation) in a fully simulated context as in the PDE approximation (S1 Fig).

All of the above need to be ran in an MPI context (e.g. `mpirun -n ... ./code/bin/dap-mpi ...`)

The implementations above are all relatively specialized for the investigations in the associated paper. However, the individual classes in `dual_adaptation_process.h`, `dynamic_dap.h`, `logistic_dap.h` and `static_dap.h` could be reused in a more general context with relative ease.

## USE

As the tools share a lot of inputs and outputs, they have been tied together in a toolchain using Snakemake. 

All the workflows start by fitting some rate function parameters to a set of observations of some hypothetical resistance mechanism. The resulting data is either directly visualized, or fed into more advanced simulators for further analysis

Snakemake workflows exist for the following files. (Note, the `up` and `down` is replaced with the name of the tables of observations)

`results/figures/up.down.abc-diagnostic.pdf`  
`results/figures/up.down.abc-fit.pdf`  
These generate plots showing diagnostic output from the ABC fitting, and some plots showing the fitted result (Fig 3)

`results/figures/up.down.verify.logistic.pdf`  
`results/figures/up.down.verify.static.pdf`  
These workflows compare the PDE output to the results of a proper stochastic simulation of cells with a continuous resistance mechanism as described in the paper (S1 Fig)

`results/figures/up.down.mpi.pdf`  
This workflow estimates the risk of mutation by running a large number of stochastic simulations. Due to the high computational cost, this probably needs to be run on an HPC cluster.

`results/figures/up.down.holiday-processed.pdf`  
`results/figures/up.down.holiday-input.pdf`  
`results/figures/up.down.holiday.pdf`  
These workflows estimate the risk of mutation for different drug-holiday scenarios. As above, the high comuptational cost mean they likely need to run on an HPC cluster. All three rely on the same simulation output however, so the simulation only needs to run once (Fig 6, 7).

To investigate new scenarios outside of those examined in the paper, one needs to provide new observation files, and to update the `simulation-config`
Let's start by inspecting the observation input files:

In [2]:
import pandas as pd

df_up = pd.read_csv('../data/raw_internal/gradual-up-4.csv')
df_down = pd.read_csv('../data/raw_internal/default-down-4.csv')

display(df_up)
display(df_down)

,time,param,stdev
0,0,1.0,1
1,25,1.1,1
2,75,1.9,1
3,100,2.0,1


,time,param,stdev
0,0,2.0,1
1,5,1.9,1
2,15,1.1,1
3,20,1.0,1


The given tables should contain fitting points for the acquisition and loss of the secondary resistance mecahism, given in the tables as the growth rate `param`, with stdev `stdev` at timepoints (in days) `time`.

We then need to set up some appropriate simulation parameters, given by the simulation-config.toml files.

In [12]:
with open('../simulation-config-4.toml', 'r') as f:
    config = f.read()

print(config)

time_points_up = 512
time_range_up = [ 0, 100,]
time_points_down = 128
time_range_down = [ 0, 20,]
parameter_points = 998
parameter_range = [ -5, 7,]
optimum_normal = 1.0
optimum_treatment = 2.0
abc_limits_shape = [ 0.1, 80.0,]
abc_limits_center = [ 0.05, 0.95,]
abc_limits_width = [ 0.1, 10.0,]
abc_limits_noise_sigma = [ 0.01, 5.0,]
abc_limits_normal_maximum_rate = [ 0.0, 5.0,]
abc_limits_maximum_rate_ratio = [ 0.01, 1.0,]
abc_min_epsilon = 0.1
abc_max_populations = 20
abc_min_acceptance = 0.0
abc_parallel_simulations = 16
abc_initial_population_size = 2048
abc_population_size_epsilon = 0.15
abc_population_size_maximum = 2048
abc_convolution_method = "fft"
holiday_time_up_factor = 4
holiday_start_range = [ 1, 768, 36,]
holiday_duration_range = [ 1, 144, 4,]
mpi_holiday_timelines = 792
mpi_holiday_simulations_per_timeline = 1000
mpi_statics_number_of_simulations = 100
mpi_statics_interaction_death_rate = 2e-5
mpi_statics_population_size = 8000
mpi_simulations_per_time_point = 100000
mpi

Let's go through each of the parameters in detail:

`time_points_up = 512`  
The PDE calculation discretizes time into this many timesteps. More steps could improve accuracy, at the cost of speed.

`time_range_up = [ 0, 100,]`  
The time range in days considered for the acquisition of the secondary resistance mechanism. Should match the datapoints in the input files (note that they also range from 0 to 100 above)

`time_points_down = 128`  
`time_range_down = [ 0, 20,]`  
Same, but for the down (i.e. loss of secondary resistance mechanism) data. Note that the down data ranges from 0 to 20 above.

`parameter_points = 998`  
Number of points the growth rate parameter is discretized to for the PDE calculation.

`parameter_range = [ -5, 7,]`  
Maximum range considered for the growth rate parameter. Note that the resolution in the parameter dimension (for an individual simulation) in this case is about 0.012 (i.e. range / points). In general, the range should be significantly wider than the observations to avoid clipping effects.

`optimum_normal = 1.0`  
`optimum_treatment = 2.0`  death_ratepted and fully adapted limits, it is reasonable to use those limits as a guess for these parameters.

`abc_limits_shape = [ 0.1, 80.0,]`  
`abc_limits_center = [ 0.05, 0.95,]`  
`abc_limits_width = [ 0.1, 10.0,]`  
`abc_limits_noise_sigma = [ 0.01, 5.0,]`  
`abc_limits_normal_maximum_rate = [ 0.0, 5.0,]`  
`abc_limits_maximum_rate_ratio = [ 0.01, 1.0,]`  
Limits to fitting parameters. Refer to figure 2 in the article for an explanation of the first three. In general, if the results are close to any limit, consider increasing the range as it might be affecting the simulation. These parameters are already quite generous. Note that the noise_sigma is the rate of 'mutation' of the continuous parameter.

`abc_min_epsilon = 0.1`  
`abc_max_populations = 20`  
`abc_min_acceptance = 0.0`  
`abc_parallel_simulations = 16`  
`abc_initial_population_size = 2048`  
`abc_population_size_epsilon = 0.15`  
`abc_population_size_maximum = 2048`  
These are simulation parameters that affect the ABC fitting procedure itself. These parameters correspond to pyABC features, so refer to its documentation for details.

`abc_convolution_method = "fft"`  
The convolution in the PDE can be calculated with either numpy.convolve `"np"` or with scipy.signal.fft `"fft"`. The latter is much faster in my testing, but that could vary depending on different parameters.

`holiday_time_up_factor = 4`  
For the drug holiday calculations, we may want to use a higher time-resolution of the secondary resistance parameter than for ABC fitting. The number of sample points is simply multiplied by this number.

`holiday_start_range = [ 1, 768, 36,]`  
`holiday_duration_range = [ 1, 144, 4,]`
The range (literally python `range(start, end, step)`) of values to test for drug holiday start-day and duration.  

`mpi_holiday_timelines = 792`  
_AUTOMATICALLY GENERATED_ The total number of start/duration combinatioons to test. This is automatically written to the parameters file by plots.py/generate-dataset-holiday.

`mpi_holiday_simulations_per_timeline = 1000`  
How many times each start/duration combination is tested. A higher number -> better statistics, and is proportional to compute time.

`mpi_statics_number_of_simulations = 100`  
`mpi_statics_interaction_death_rate = 2e-5`  
`mpi_statics_population_size = 8000`  
Parameters for a simple stochastic simulation to verify that a stochastic simulation with the ABC/PDE derived parameters matches the approximate results (S1 Fig)

`mpi_cores_per_node = 2`  
Number of threads spawned within each worker for any of the MPI calculations. Pick a value depending on the cluster hardware.

`mpi_max_population_size = 1000`  
`mpi_max_time = 10000`  
Used for both the holiday and the risk of mutation simulations. The max population size is size a mutant population needs to reach to be considered established and not at an immediate risk of randomly dying out. Max time is also included as an alternate stopping condition, to prevent neverending simulations in certain scenarios.

`mpi_simulations_per_time_point = 100000`  
`mpi_variable_death_rate = 1`  
Parameters for evaluating the risk of mutations.  The `variable_death_rate` setting (bool, 0 or 1) enables the feature where the mutant growth rate is the death rate of the normal cells at that time, as discussed in the paper.

`mpi_death_rate = 0.1021828563162995`  
`mpi_noise_function_sigma = 0.3175062611664512`  
`mpi_rate_function_center = 0.5816861905086594`  
`mpi_rate_function_shape = 53.466783970775545`  
`mpi_rate_function_width = 1.3024316336028647`  
`mpi_rate_function_max = 1.9100680613756666`  
`mpi_rate_function_ratio = 0.18677983926546635`  
_AUTOMATICALLY GENERATED_ These are all automatically written by plots.py/generate-dataset-mpi or plots.py/generate-dataset-verify, and are parameters derived from the ABC fitting to be fed into proper stochastic simulations.

`mpi_rng_seeds = [ -950663638, 748972135, 375983106, ..., ]`  
Random number generation seeds that are provided to the MPI processes for reproducibility (shortened here for space).

## HOW TO USE NEW INPUT
As stated, there are three things that need to be provided to test a new scenarios.

Observations for the gain and loss of the resistance mechanism need to be provided like the `.csv` tables shown earliers. `param` can be normalized arbitrarily, though, a value of 1 for the untreated optimum, a value > 1 for the treated optimum, and a reasonably conservative total range, is recommended. The `.csv` files should be placed in `data/raw_internal` like the provided files.

The settings need to be adjusted to work for that input. In particular:
- update `time_range_*` to match the new observations
- update `parameter_range`, `optimum_normal` and `optimum_treatment` to match the low and high observations.

With that, running
```sh
snakemake results/figures/[up].[down].abc-diagnostic.pdf
snakemake results/figures/[up].[down].abc-fit.pdf
```
to fit the rate function parameters to the observations, and optionally
```sh
snakemake results/figures/[up].[down].verify.logistic.pdf
snakemake results/figures/[up].[down].verify.static.pdf
```
to verify that the fit holds up in a detailed simulation, is possible.
`[up]` and `[down]` should be replaced with the name of the up and down observation file (sans .csv) that are used.

After verifying that the results look good (e.g. check the abc-diagnostic for convergence and lack of clipping for any parameter, check verify that proper simulation agrees with approximations) one may proceed to estimating the risk of resistance mutation. If abc hasn't converged, or there is clipping, try adjusting the corresponding parameters and rerunning the above commands.

Estimating the risk of mutation as a function of time likely doesn't need any further adjustments to the settings file. We can simply run
```sh
snakemake results/figures/[up].[down].mpi.pdf
```
and though it can run on a regular computer, it may be necessary to use some larger (MPI capable) cluster in the interest of time. Some clusters may struggle with rendering graphics for the pdf. `rule mpi` (see the Snakefile) is the computationally heavy step, so, a workaround is to run the steps leading up to that command locally, run that on a cluster, and then run the last step locally again.

The details of running the workflow in an HPC context is going to vary between different clusters, so cannot easily be described here. However, the most flexible way is very likely to write a script that runs the command in the `rule mpi` rule and to specifically copy those input files to the cluster, rather than to rely on it to support snakemake. But as stated, it will depend, and if running on some larger workstation/server rather than a shared cluster then snakemake is more likely the easier option.

For the drug holiday simulations, first adjust `holiday_start_range` and `holiday_duration_range` to some values that seem reasonable in light of the resistance adaptation curves in `*.abc-fit.pdf`. Try to effectively cover the range where adaptation is happening, as outside of that the results are going to be constant. The other parameters can be left as-is. To generate all the results run:
```sh
snakemake results/figures/[up].[down].holiday-processed.pdf
snakemake results/figures/[up].[down].holiday-input.pdf
snakemake results/figures/[up].[down].holiday.pdf
```
Note that same as above, only one step in the pipeline `rule holiday` is actually costly enough to (more or less) require an HPC cluster, and the considerations apply.

## HACKING THE SIMULATORS
For implementing more unique simulations outside the scope of this paper, the simulation classes could be reused. `dual_adaptation_process.h` is probably the best starting point, as it has an unconstrained population size and no special features. The code has been generously commented so should be comparatively easy to adapt.

The most particular feature is that the simulator is templated on a rate function functor (i.e. an object that implements operator()(double) -> double). As an example, here is the rate function described in the paper (as used in the mpi simulators). By templating like this, the simulation engine can support parametrizeable rate functions without having to alter the simulatino engine source code.
```C++
struct RateBeta {
  RateBeta(double s, double c, double w, double u, double m)
    : a(s*c)
    , b(s - a)
    , c(c)
    , w(w)
    , u(u)
    , m(m) {
    factor = pow(a, a) * pow(b, b) * pow(a+b, -(a + b));
  }
  double a, b, c, w, u, m;
  double factor;
  double operator()(double x) {
    if (x <= u - c*w || x >= u - (c - 1)*w)
      return 0.0;
    double y = pow(x/w - u/w + c, a) * pow(1 - (x/w - u/w + c), b);
    y = m*y/factor;
    return y;
  }
};
```